In [4]:
#Testing DSA OCR service features
import girder_client, os,sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    

import  ocr
import server
from IPython.display import Image as IPImage
    
    
# import PIL
# from PIL import Image
import requests
%load_ext autoreload
%autoreload 2

In [5]:
gc=girder_client.GirderClient(apiUrl="http://transplant.digitalslidearchive.emory.edu:8080/api/v1")
gc.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


In [7]:
## 5cb8e68ae629140067608a4c
## Folders
import tqdm

alreadyHadDataMatrix = 0
batchSize = 10


for f in gc.listFolder('5cb8e389e6291400676076bf',parentFolderType='collection'):
    print("Scanning folder %s" % f['name'])
    IDS_to_Process = []

    for i in tqdm.tqdm(list(gc.listItem(f['_id']))):
        if ('meta' in i):
            if ('dataMatrixOutput' in i['meta']):
                alreadyHadDataMatrix+=1
                #print ("Meta Tag alraedy exists")

        else:
            IDS_to_Process.append(i['_id'])
            if(len(IDS_to_Process)%batchSize == 0):
                print("You have a batch of %s"%batchSize)
                labelUpdateUrl = "http://localhost:5000/label?ids=%s" % ( ",".join(IDS_to_Process))
                print(labelUpdateUrl)
                requests.get(labelUpdateUrl)
                IDS_to_Process = [] ### Reset the cache


    ## Don't forget to process the last batch
    if(len(IDS_to_Process)>0):
        
        labelUpdateUrl = "http://0.0.0.0:5000/label?ids=%s" % ( ",".join(IDS_to_Process))
        print(labelUpdateUrl)
        requests.get(labelUpdateUrl)

    print(alreadyHadDataMatrix,"Had a data matrix alraedy")


    

100%|██████████| 277/277 [00:00<00:00, 1305418.21it/s]

Scanning folder 2019-03-14
http://0.0.0.0:5000/label?ids=5cb8e67be62914006760898f
358 Had a data matrix alraedy
Scanning folder 2019-03-15
635 Had a data matrix alraedy
Scanning folder 2019-03-16



100%|██████████| 306/306 [00:00<00:00, 1304326.24it/s]


790 Had a data matrix alraedy
Scanning folder 2019-03-18
http://0.0.0.0:5000/label?ids=5cb8e7a8e629140067609344,5cb8e7bbe6291400676093d4,5cb8e816e6291400676096bf,5cb8e83ce6291400676097d4
1092 Had a data matrix alraedy
Scanning folder 2019-03-19


100%|██████████| 382/382 [00:00<00:00, 929364.34it/s]


...

You have a batch of 10
http://localhost:5000/label?ids=5cb8e473e629140067607977,5cb8e439e62914006760785f,5cb8e472e629140067607975,5cb8e453e6291400676078d7,5cb8e473e62914006760797b,5cb8e416e62914006760776d,5cb8e467e62914006760793f,5cb8e474e62914006760797d,5cb8e473e629140067607979,5cb8e474e62914006760797f


100%|██████████| 265/265 [07:45<00:00,  1.75s/it]


http://0.0.0.0:5000/label?ids=5cb8e416e62914006760776f,5cb8e463e62914006760792d,5cb8e43ae629140067607863,5cb8e43ae629140067607861,5cb8e43be629140067607865


0it [00:00, ?it/s]
0it [00:00, ?it/s]

2844 Had a data matrix alraedy
Scanning folder Bela
2844 Had a data matrix alraedy
Scanning folder Tubulitis_anotation
2844 Had a data matrix alraedy


In [ ]:
img = server.get_image(gc,i['_id'],label=True)

In [ ]:
r,r,md = ocr.basic_ocr(img)


In [ ]:
print("http://0.0.0.0:5000/label?ids=%s" % ( ",".join(IDS_to_Process)))

In [ ]:
#curl -X GET --header 'Accept: image/jpeg' --header 'Girder-Token: GOL44tmAFOVRBzzBJApVRrfUh87CQkh7CGPuvsBwSnmUHsrWQoig4PjUPzSCmL4H' 'http://transplant.digitalslidearchive.emory.edu:8080/api/v1/'

imgId = '5cb8e742e62914006760903a'

%time labelImg = gc.get('item/%s/tiles/images/label?encoding=JPEG&width=256'%imgId,jsonResp=False)

In [ ]:
IPImage(labelImg.content)

In [ ]:
import numpy as np
from PIL import Image
import numpy as np

import cv2


img_array = np.frombuffer(labelImg.content, dtype=np.uint8)
img = cv2.imdecode(img_array, cv2.IMREAD_COLOR)


procImg = img
#cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

(h,w,d) = procImg.shape
wStart = int(w*0.05)
wEnd = int(w*0.8)
hStart = int(h*0)
hEnd = int(h*0.55)

print(h,w,d)
procImg = procImg[50:400,50:600,]

print(wStart,wEnd,hStart,hEnd)

%time result = ocr.pytesseract.image_to_string(procImg)
print(result)

Image.fromarray(procImg)



In [ ]:
%time ocr.decode(procImg)

In [ ]:
from pylibdmtx.pylibdmtx import decode
from PIL import Image
decode(labelImg.content)